<a href="https://colab.research.google.com/github/noeyxos/Python/blob/main/PythonAzure01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install azure-storage-blob azure-identity            # azure blob storage를 다루는 패키지와 identity 인증을 하는 패키지 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os, uuid       
from azure.identity import DefaultAzureCredential                               #Credential [에버랜드 손목띠 - 한번 인증하고 나면 계속 남아있는 거]
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# Blob Services 에 접속
connect_str = ''                                                            # 블롭 스토리지에서 연결 문자열 가져오기
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [ ]:
container_name = 'test2container'             #새로운 컨테이너 만들어보기

container_client = blob_service_client.create_container(container_name)

In [ ]:
# Create a local directory to hold blob data
local_path = './data'                 # 현재 경로 밑에 data 폴더 만들기위해 경로 지정
os.mkdir(local_path)                # os 모듈이 mkdir(local_path): local_path 디렉토리를 만드는 메소드 

# Create a file in the local data directory to upload and download
# 파일 하나의 이름 
local_file_name ='welcome' + '.txt'    
# 파일 전체의 경로 
upload_file_path = os.path.join(local_path, local_file_name)      #join(): 디렉토리와 파일이름을 합쳐서 하나의 경로로 만들어 줌   


#Write text to the file 
file = open(upload_file_path, mode='w')       # mode='w' 쓰기 모드로 열기 
file.write('Welcome to Python')                      # 파일 쓰기
file.close()


In [ ]:
# blob storage의 정보 가져오기
blob_client = blob_service_client.get_blob_client(
    container = container_name,
    blob = local_file_name
)

with open(file = upload_file_path, mode='rb') as data:                          #'rb' : 읽기전용 binary 파일   
  blob_client.upload_blob(data)                                                              # blob container에 바로 올림 (파일이 하나도 안깨지는 장점)    

In [ ]:
# blob 파일의 리스트를 확인한다. 
print('\nListing blobs')

blob_list = container_client.list_blobs()       #container_client에 list_blobs를 가져와서 
for blob in blob_list:                                    #blob 을 확인하자
  print('\t'+ blob.name)


Listing blobs
	welcome.txt


In [ ]:
# Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extendsion so you can see both files in the data direct

#blob storage에 업로드
download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))           #str.replace(타겟, 바꾸기 전, 바뀐 후) : local_file_name의 문자열을 바꾸어줌
print(download_file_path)

#blob_service 야 container를 가져와라 ! 
container_client = blob_service_client.get_container_client(container = container_name)    #container의 정보를 가져옴

#download
with open(file=download_file_path, mode='wb') as download_file:                    # download_file로 파일 열고
  download_file.write(container_client.download_blob(blob.name).readall())      # container에서 blob 을 다 읽어와서 다운, 그리고 download_file에 쓰기 

./data/welcome_DOWNLOAD.txt


작업 끝나고 나서 리소스 정리할 때 사용

In [ ]:
# clean Up (삭제) : 작업 끝나고 나서 리소스를 정리할 때 사용
print('\nPress the Enter Key to begin clean up')    # 경고 ! 엔터키를 누르면 다 지워진다! 
input()

print('Deleting blob container...')
container_client.delete_container()

print('Deleting the local source and downloaded files...')
os.remove(upload_file_path)                                             
os.remove(download_file_path)
os.rmdir(local_path)                                                            #rmdir : removedirectory 

print('Done')



Press the Enter Key to begin clean up

Deleting blob container...
Deleting the local source and downloaded files...
Done


In [ ]:
# 이번에는 yes를 입력하면  clean Up (삭제) 되는 코드 작성
print('\n Write "yes" to begin clean up')    # 경고 ! 엔터키를 누르면 다 지워진다! 
if input() == 'yes' :

  print('Deleting blob container...')
  container_client.delete_container()

  print('Deleting the local source and downloaded files...')
  os.remove(upload_file_path)                                             
  os.remove(download_file_path)
  os.rmdir(local_path)                                                            #rmdir : removedirectory 

  print('Done')